In [1]:
from cassandra.cluster import Cluster
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('data_stock')

In [3]:
session.set_keyspace('data_stock')

In [ ]:
rows_price = session.execute("""
    SELECT * FROM candlestick1_1068_data ALLOW FILTERING
""")
df_cand = pd.DataFrame(rows_price)
df_cand['time'] = pd.to_datetime(df_cand['time'])
df_cand = df_cand.sort_values(['symbol', 'time'])

In [10]:
# ✅ แปลงคอลัมน์ time ให้เป็น datetime (ถ้ายังไม่ได้แปลง)
df_cand['time'] = pd.to_datetime(df_cand['time'])

# ✅ เลือกเฉพาะตั้งแต่ 1 มิถุนายน 2025 เป็นต้นไป
df_2025= df_cand[df_cand['time'] >= '2025-06-01']

In [11]:
df_2025.to_excel("stock_june2025_onwards.xlsx", index=False)

In [19]:
df_2025_2=pd.read_excel("stock_june2025_onwards.xlsx")

In [21]:
df_2025_2.head(10)

,symbol,time,close_price,high_price,low_price,open_price,value,volume
0,24CS,2025-06-04,0.94,0.98,0.90,0.91,0,13587837
1,24CS,2025-06-05,0.96,1.02,0.93,0.94,0,21894535
2,24CS,2025-06-06,1.02,1.04,0.95,0.96,0,29946221
3,24CS,2025-06-09,1.20,1.28,1.06,1.10,0,73692416
4,24CS,2025-06-10,1.45,1.55,1.22,1.24,0,91965466
5,24CS,2025-06-11,1.33,1.44,1.28,1.31,0,22963184
6,24CS,2025-06-12,1.25,1.35,1.15,1.32,0,16676458
7,24CS,2025-06-13,1.23,1.31,1.18,1.20,0,14148120
8,24CS,2025-06-16,1.40,1.40,1.20,1.23,0,25850193
9,24CS,2025-06-17,1.40,1.46,1.32,1.37,0,21305151


In [20]:
df_2025_2.tail(10)

,symbol,time,close_price,high_price,low_price,open_price,value,volume
69352,ZIGA,2025-09-17,1.26,1.29,1.26,1.29,0,2470802
69353,ZIGA,2025-09-18,1.23,1.27,1.23,1.27,0,2274003
69354,ZIGA,2025-09-19,1.26,1.27,1.24,1.24,0,2737000
69355,ZIGA,2025-09-22,1.24,1.32,1.24,1.26,0,14527172
69356,ZIGA,2025-09-23,1.20,1.25,1.18,1.24,0,2827412
69357,ZIGA,2025-09-24,1.19,1.21,1.17,1.20,0,1570200
69358,ZIGA,2025-09-25,1.19,1.21,1.18,1.20,0,1031400
69359,ZIGA,2025-09-26,1.20,1.20,1.18,1.19,0,896500
69360,ZIGA,2025-09-29,1.18,1.21,1.17,1.21,0,1702800
69361,ZIGA,2025-09-30,1.17,1.19,1.16,1.18,0,1464271


In [14]:
rows_fin2 = session.execute("""
    SELECT symbol,eps,pe,pbv,percentYield FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_fun= pd.DataFrame(rows_fin2)
df_fun.head(10)

,symbol,eps,pe,pbv,percentyield
0,PPPM,0.02000,0.00,0.70,0.00
1,TPCH,0.24112,7.90,0.34,13.91
2,KPNREIT,NaN,NaN,0.23,0.00
3,POLY,0.27632,12.89,2.42,6.38
4,QHBREIT,NaN,NaN,0.41,0.00
5,VCOM,0.21428,7.85,1.48,10.79
6,KDH,4.12186,10.62,1.93,3.01
7,NVD,-0.00942,38.87,0.32,0.00
8,JDF,0.04224,12.19,1.35,4.85
9,SVR,-0.03402,0.00,0.34,0.00


In [15]:
# ✅ 1) ดึงแท่งล่าสุดของแต่ละ symbol
df_latest_candle = (
    df_cand.sort_values("time")
    .groupby("symbol", as_index=False)
    .tail(1)  # หรือใช้ .last() ก็ได้หลัง sort แล้ว
)

# ✅ 2) Merge กับข้อมูล fundamental
df = pd.merge(df_fun, df_latest_candle, on="symbol", how="inner")

# ✅ 3) คำนวณ marketcap
df["marketcap"] = df["close_price"] * df["volume"]

In [16]:
df.head()

,symbol,eps,pe,pbv,percentyield,time,close_price,high_price,low_price,open_price,value,volume,marketcap
0,PPPM,0.02000,0.00,0.70,0.00,2025-09-30,0.40,0.43,0.39,0.42,0.0,263750,1.055000e+05
1,TPCH,0.24112,7.90,0.34,13.91,2025-09-30,2.84,2.90,2.84,2.86,0.0,141400,4.015760e+05
2,KPNREIT,NaN,NaN,0.23,0.00,2025-09-30,2.10,2.10,2.10,2.10,0.0,100,2.100000e+02
3,POLY,0.27632,12.89,2.42,6.38,2025-09-30,6.80,7.05,6.80,6.90,0.0,22400,1.523200e+05
4,QHBREIT,NaN,NaN,0.41,0.00,2025-09-30,3.68,3.72,3.66,3.68,0.0,520738,1.916316e+06


In [17]:
df_fun2 = df.drop(columns=['time',
       'close_price', 'high_price', 'low_price', 'open_price', 'value'])

In [18]:
df_fun2.to_excel("stock_fun.xlsx", index=False)

In [22]:
df_fun=pd.read_excel("stock_fun.xlsx")
df_fun.head()

,symbol,eps,pe,pbv,percentyield,volume,marketcap
0,PPPM,0.02000,0.00,0.70,0.00,263750,1.055000e+05
1,TPCH,0.24112,7.90,0.34,13.91,141400,4.015760e+05
2,KPNREIT,NaN,NaN,0.23,0.00,100,2.100000e+02
3,POLY,0.27632,12.89,2.42,6.38,22400,1.523200e+05
4,QHBREIT,NaN,NaN,0.41,0.00,520738,1.916316e+06


In [23]:
df_fun.shape

(907, 7)